# Proyecto final Técnicas de inteligencia Artificial.
Integrantes:
- Leonardo Fabio Mercado Benítez.
- María Alejandra Arias Torres.

In [5]:
import cv2
import numpy as np
import pyautogui as pg
print('Librerias importadas correctamente')

Librerias importadas correctamente


Intento 1

In [2]:
video = cv2.VideoCapture(0)

i = 0
kernel = np.ones((5, 5), np.uint8)

while True:
    ret, frame = video.read()
    if ret == False:
        break
    cv2.imshow("Color", frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if i == 20:
        bgGray = gray
    elif i > 20:
        dif = cv2.absdiff(gray, bgGray)
        cv2.imshow('Diferencia', dif)
        _, th = cv2.threshold(dif, 40, 255, cv2.THRESH_BINARY)
        cv2.imshow("Umbralizado", th)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        cv2.imshow("pre-Filtrado", closing)
        
    i = i + 1
        

    if cv2.waitKey(30) & 0xFF == ord("q"):
        break
        
video.release()
cv2.destroyAllWindows();

intento 2

In [3]:
cap = cv2.VideoCapture(0)

bg = None #Flag del fondo de la imagen
kernel = np.ones((5, 5), np.uint8)

while (True):
    ret, frame = cap.read()    
    if (ret == False): break
    
    frameaux = frame.copy()
    
    if bg is not None:
        
        #cv2.imshow('fondo',bg)
        ROI = frame[:,:]
        cv2.rectangle(frame,(285-2,75-2),(5+2,400+2),(0,255,255))
        grayROI = cv2.cvtColor(ROI,cv2.COLOR_BGR2GRAY)
        bgROI = bg[:,:]
        dif = cv2.absdiff(grayROI,bgROI)
        #cv2.imshow('diferencia',dif)
        _, th = cv2.threshold(dif,30,255,cv2.THRESH_BINARY)
        #cv2.imshow('umbralizado',th)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        #cv2.imshow("pre-Filtrado", closing)
        th = cv2.medianBlur(closing,7)
        cv2.imshow("Filtrado", th)
        
        #Hallamos los contornos
        cnts, _ = cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#SE hallan los contornos
        cnts = sorted(cnts, key=cv2.contourArea,reverse=True)[:1] #Se organizan los contornos por áreas
        #Graficamos los contornos.
        cv2.drawContours(ROI,cnts,0,(0,255,0),2)
        
        for cnt in cnts:
            #Hallar el centro del área
            M = cv2.moments(cnt)
            if M["m00"] == 0: M["m00"]=1
            x = int(M["m10"]/M["m00"])
            y = int(M["m01"]/M["m00"])
            #Gráficamos el centro
            cv2.circle(ROI,tuple([x,y]),5,(0,0,255),-1)
            #print(f'la mano está en X = [{x}] y en Y = [{y}]')
            
            #Hallar y mostrar el punto más alto del área
            ymin = cnt.min(axis=1)
            cv2.circle(ROI,tuple(ymin[0]),5,(0,255,255),-1)
            
            #Hallar y gráfica el casco convexo
            hull1 = cv2.convexHull(cnt)
            cv2.drawContours(ROI, [hull1],0,(0,0,255),2)
            
            hull2 = cv2.convexHull(cnt,returnPoints=False)
            defects = cv2.convexityDefects(cnt,hull2)
            
            if defects is not None:
                
                inicio = []
                fin = []
                fingers = 0
                
                for i in range(defects.shape[0]):
                    
                    s,e,f,d = defects[i,0]
                    start = cnt[s][0]
                    end = cnt[e][0]
                    far = cnt[f][0]
                    
                    #Cálculo de distancias
                    a = np.linalg.norm(far - end)
                    b = np.linalg.norm(far - start)
                    c = np.linalg.norm(start - end)
                    #Cálculo de ángulos
                    angulo = np.arccos((np.power(a,2)+np.power(b,2)-np.power(c,2))/(2*a*b))
                    angulo = np.degrees(angulo)
                    angulo = int(angulo)
                    
                    if np.linalg.norm(start-end) > 40 and d > 12000 and angulo < 90:
                        inicio.append(start)
                        fin.append(end)
                        #cv2.putText(ROI,'{}'.format(d),tuple(far),1,1,(0,255,0),1,cv2.LINE_AA)  #Se imprime la distancia entre el far y el start                                                      
                        #cv2.putText(ROI,'{}'.format(angulo),tuple(far),1,1,(0,0,0),1,cv2.LINE_AA) # Se imprime el angulo más grande entre el start far y end                                                  
                        #cv2.circle(ROI,tuple(start),5,(0,0,0),2)
                        #cv2.circle(ROI,tuple(end),5,(255,0,0),2)
                        #cv2.circle(ROI,tuple(far),7,(255,255,255),2)
                        
                        
                if len(inicio) == 0:
                    miny = np.linalg.norm(ymin[0]-[x,y])
                    if miny >= 100:
                        fingers = fingers + 1
                        cv2.putText(ROI,'{}'.format(fingers),tuple(ymin[0]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                for i in range(len(inicio)):
                    fingers = fingers + 1;
                    cv2.putText(ROI,'{}'.format(fingers),tuple(inicio[i]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                    if i == len(inicio) - 1: 
                        fingers = fingers + 1
                        cv2.putText(ROI,'{}'.format(fingers),tuple(fin[i]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                cv2.putText(ROI,'{}'.format(fingers),tuple([280,70]),1,2.5,(0,0,255),2,cv2.LINE_AA)
                                
    cv2.imshow('conteo',frame)
    cv2.imshow('principal',frameaux)
    
    k = cv2.waitKey(20)
    if (k== ord('b')):
        bg = cv2.cvtColor(frameaux,cv2.COLOR_BGR2GRAY)
    if k == 27: break 


cap.release()
cv2.destroyAllWindows()

In [4]:
video.release()
cv2.destroyAllWindows();

NameError: name 'video' is not defined

Intento 3

In [4]:
cap = cv2.VideoCapture(0)

bg = None #Flag del fondo de la imagen
kernel = np.ones((5, 5), np.uint8)

while (True):
    ret, frame = cap.read()    
    if (ret == False): break
    
    frameaux = frame.copy()
    
    if bg is not None:
        
        #cv2.imshow('fondo',bg)
        ROI = frame[:,:]
        cv2.rectangle(frame,(285-2,75-2),(5+2,400+2),(0,255,255))
        grayROI = cv2.cvtColor(ROI,cv2.COLOR_BGR2GRAY)
        bgROI = bg[:,:]
        dif = cv2.absdiff(grayROI,bgROI)
        #cv2.imshow('diferencia',dif)
        _, th = cv2.threshold(dif,30,255,cv2.THRESH_BINARY)
        #cv2.imshow('umbralizado',th)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        #cv2.imshow("pre-Filtrado", closing)
        th = cv2.medianBlur(closing,7)
        cv2.imshow("Filtrado", th)
        
        #Hallamos los contornos
        cnts, _ = cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#SE hallan los contornos
        cnts = sorted(cnts, key=cv2.contourArea,reverse=True)[:1] #Se organizan los contornos por áreas
        #Graficamos los contornos.
        cv2.drawContours(ROI,cnts,0,(0,255,0),2)
        
        for cnt in cnts:
            #Hallar el centro del área
            M = cv2.moments(cnt)
            if M["m00"] == 0: M["m00"]=1
            x = int(M["m10"]/M["m00"])
            y = int(M["m01"]/M["m00"])
            #Gráficamos el centro
            cv2.circle(ROI,tuple([x,y]),5,(0,0,255),-1)
            #print(f'la mano está en X = [{x}] y en Y = [{y}]')
            
            #Hallar y mostrar el punto más alto del área
            ymin = cnt.min(axis=1)
            cv2.circle(ROI,tuple(ymin[0]),5,(0,255,255),-1)
            #print(f'El punto más alto del dedo es = {ymin}')
            
            #Hallar y gráfica el casco convexo
            hull1 = cv2.convexHull(cnt)
            cv2.drawContours(ROI, [hull1],0,(0,0,255),2)
            
            hull2 = cv2.convexHull(cnt,returnPoints=False)
            defects = cv2.convexityDefects(cnt,hull2)
            
            if defects is not None:
                
                inicio = []
                fin = []
                fingers = 0
                
                for i in range(defects.shape[0]):
                    
                    s,e,f,d = defects[i,0]
                    start = cnt[s][0]
                    end = cnt[e][0]
                    far = cnt[f][0]
                    
                    #Cálculo de distancias
                    a = np.linalg.norm(far - end)
                    b = np.linalg.norm(far - start)
                    c = np.linalg.norm(start - end)
                    #Cálculo de ángulos
                    angulo = np.arccos((np.power(a,2)+np.power(b,2)-np.power(c,2))/(2*a*b))
                    angulo = np.degrees(angulo)
                    angulo = int(angulo)
                    
                    if np.linalg.norm(start-end) > 40 and d > 12000 and angulo < 90:
                        inicio.append(start)
                        fin.append(end)
                        #cv2.putText(ROI,'{}'.format(d),tuple(far),1,1,(0,255,0),1,cv2.LINE_AA)  #Se imprime la distancia entre el far y el start                                                      
                        #cv2.putText(ROI,'{}'.format(angulo),tuple(far),1,1,(0,0,0),1,cv2.LINE_AA) # Se imprime el angulo más grande entre el start far y end                                                  
                        #cv2.circle(ROI,tuple(start),5,(0,0,0),2)
                        #cv2.circle(ROI,tuple(end),5,(255,0,0),2)
                        #cv2.circle(ROI,tuple(far),7,(255,255,255),2)
                        
                        
                if len(inicio) == 0:
                    miny = np.linalg.norm(ymin[0]-[x,y])
                    if miny >= 100:
                        fingers = fingers + 1
                        cv2.putText(ROI,'{}'.format(fingers),tuple(ymin[0]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                for i in range(len(inicio)):
                    fingers = fingers + 1;
                    cv2.putText(ROI,'{}'.format(fingers),tuple(inicio[i]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                    if i == len(inicio) - 1: 
                        fingers = fingers + 1
                        cv2.putText(ROI,'{}'.format(fingers),tuple(fin[i]),1,1.5,(255,0,0),1,cv2.LINE_AA)
                cv2.putText(ROI,'{}'.format(fingers),tuple([280,70]),1,2.5,(0,0,255),2,cv2.LINE_AA)
                
                
                #------------------------------------------------------------------------------
                #Para manejar el mouse
                if fingers == 1:
                    x = ymin[0][0]
                    y = ymin[0][1]
                    pg.moveTo(364 + ymin[0][0]*3,51 + ymin[0][1]*3)
                elif(fingers == 0):
                    pg.moveTo(634,51)     
                elif(fingers == 5):
                    pg.click()
                #------------------------------------------------------------------------------
                
    cv2.imshow('conteo',frame)
    cv2.imshow('principal',frameaux)
    
    k = cv2.waitKey(20)
    if (k== ord('b')):
        bg = cv2.cvtColor(frameaux,cv2.COLOR_BGR2GRAY)
    if k == 27: break 


cap.release()
cv2.destroyAllWindows()

In [12]:
video.release()
cv2.destroyAllWindows();

NameError: name 'video' is not defined